<a href="https://colab.research.google.com/github/chenrobotcat/ML100-Days/blob/master/%E3%80%8Ctfdv_basic_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 The TensorFlow Authors.

In [62]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Data Validation
***An Example of a Key Component of TensorFlow Extended***

Note: You can run this example right now in a Jupyter-style notebook, no setup required!  Just click "Run in Google Colab"

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/data_validation/tfdv_basic">
<img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.sandbox.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/data_validation/tfdv_basic.ipynb">
<img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/tfx/blob/master/docs/tutorials/data_validation/tfdv_basic.ipynb">
<img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">View source on GitHub</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/data_validation/tfdv_basic.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a></td>
</table></div>

This example colab notebook illustrates how TensorFlow Data Validation (TFDV) can be used to investigate and visualize your dataset.  That includes looking at descriptive statistics, inferring a schema, checking for and fixing anomalies, and checking for drift and skew in our dataset.  It's important to understand your dataset's characteristics, including how it might change over time in your production pipeline.  It's also important to look for anomalies in your data, and to compare your training, evaluation, and serving datasets to make sure that they're consistent.

We'll use data from the [Taxi Trips dataset](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew) released by the City of Chicago.

Note: This site provides applications using data that has been modified for use from its original source, www.cityofchicago.org, the official website of the City of Chicago. The City of Chicago makes no claims as to the content, accuracy, timeliness, or completeness of any of the data provided at this site. The data provided at this site is subject to change at any time. It is understood that the data provided at this site is being used at one’s own risk.

[Read more](https://cloud.google.com/bigquery/public-data/chicago-taxi) about the dataset in [Google BigQuery](https://cloud.google.com/bigquery/). Explore the full dataset in the [BigQuery UI](https://bigquery.cloud.google.com/dataset/bigquery-public-data:chicago_taxi_trips).

Key Point: As a modeler and developer, think about how this data is used and the potential benefits and harm a model's predictions can cause. A model like this could reinforce societal biases and disparities. Is a feature relevant to the problem you want to solve or will it introduce bias? For more information, read about [ML fairness](https://developers.google.com/machine-learning/fairness-overview/).

The columns in the dataset are:
<table>
<tr><td>pickup_community_area</td><td>fare</td><td>trip_start_month</td></tr>

<tr><td>trip_start_hour</td><td>trip_start_day</td><td>trip_start_timestamp</td></tr>
<tr><td>pickup_latitude</td><td>pickup_longitude</td><td>dropoff_latitude</td></tr>
<tr><td>dropoff_longitude</td><td>trip_miles</td><td>pickup_census_tract</td></tr>
<tr><td>dropoff_census_tract</td><td>payment_type</td><td>company</td></tr>
<tr><td>trip_seconds</td><td>dropoff_community_area</td><td>tips</td></tr>
</table>

## Install and import packages

Install the packages for TensorFlow Data Validation.

### Upgrade Pip

To avoid upgrading Pip in a system when running locally, check to make sure that we're running in Colab.  Local systems can of course be upgraded separately.

In [63]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Install Data Validation packages

Install the TensorFlow Data Validation packages and dependencies, which takes a few minutes. You may see warnings and errors regarding incompatible dependency versions, which you will resolve in the next section.

In [64]:
print('Installing TensorFlow Data Validation')
!pip install --upgrade 'tensorflow_data_validation[visualization]<2'

Installing TensorFlow Data Validation
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Import TensorFlow and reload updated packages

The prior step updates the default packages in the Gooogle Colab environment, so you must reload the package resources to resolve the new dependencies.

Note: This step resolves the dependency error from the installation. If you are still experiencing code execution problems after running this code, restart the runtime (Runtime > Restart runtime ...).

In [65]:
import pkg_resources
import importlib
importlib.reload(pkg_resources)

<module 'pkg_resources' from '/usr/local/lib/python3.7/dist-packages/pkg_resources/__init__.py'>

Check the versions of TensorFlow and the Data Validation before proceeding. 

In [66]:
import tensorflow as tf
import tensorflow_data_validation as tfdv
print('TF version:', tf.__version__)
print('TFDV version:', tfdv.version.__version__)

TF version: 2.8.2
TFDV version: 1.8.0


## Load the dataset
We will download our dataset from Google Cloud Storage.

In [67]:
import os
import tempfile, urllib, zipfile

from google.colab import files
# 上傳csv檔案
uploaded = files.upload()

# Set up some globals for our file paths
# BASE_DIR = tempfile.mkdtemp()
# print(BASE_DIR)
# DATA_DIR = os.path.join(BASE_DIR, 'data')
# OUTPUT_DIR = os.path.join(BASE_DIR, 'chicago_taxi_output')
# TRAIN_DATA = os.path.join(DATA_DIR, 'train', 'data.csv')
# EVAL_DATA = os.path.join(DATA_DIR, 'eval', 'data.csv')
# SERVING_DATA = os.path.join(DATA_DIR, 'serving', 'data.csv')

# Download the zip file from GCP and unzip it
#zip, headers = urllib.request.urlretrieve('https://storage.googleapis.com/artifacts.tfx-oss-public.appspot.com/datasets/chicago_data.zip')
#zipfile.ZipFile(zip).extractall(BASE_DIR)
#zipfile.ZipFile(zip).close()

#print("Here's what we downloaded:")
#!ls -R {os.path.join(BASE_DIR, 'data')}

Saving FS633_w2_天氣對齊.xlsx to FS633_w2_天氣對齊 (2).xlsx


In [68]:
import pandas as pd
import io

data = pd.read_excel(io.BytesIO(uploaded['FS633_w2_天氣對齊.xlsx']))

#print(data.head)

target_column = 'avg'
# 去除 'remaining','qty_105'
all_column_names = ['avg','qty_105','qty_all', 'festiva_name', 'sale_date', 'stnpres', 'seapres', 'max_temperature', 'min_temperature', 
                    'avg_temperature', 'min_rh', 'avg_rh', 'avg_wd', 'avg_wdgust', 'max_visb', 'avg_visb', 'max_uvi', 'avg_uvi', 
                    'max_cloudamount', 'stnpres-2', 'seapres-2', 'min_temperature-2', 'avg_temperature-2', 'max_tddewpoint-2', 
                    'min_tddewpoint-2', 'avg_tddewpoint-2', 'min_rh-2', 'avg_rh-2', 'max_wd-2', 'max_wdgust-2', 'avg_wdgust-2', 
                    'max_visb-2', 'min_visb-2', 'avg_visb-2']
features_column_names = all_column_names[1:]

In [69]:
train_start = 0
train_end = 593
val_start = train_end +1
val_end = val_start+297

t_start = val_end+1
t_end = t_start+360


tran_rows = val_end + 1

#print(data.head)
x_data = data[train_start:train_end]
TRAIN_DATA = pd.DataFrame(x_data, index=x_data.index, columns=features_column_names)
#print(TRAIN_DATA.head)

v_data = data[val_start:val_end]
EVAL_DATA = pd.DataFrame(v_data, index=v_data.index, columns=features_column_names)
#print(EVAL_DATA.head)

t_data = data[t_start:t_end]
SERVING_DATA =  pd.DataFrame(t_data, index=t_data.index, columns=features_column_names)

## Compute and visualize statistics

First we'll use [`tfdv.generate_statistics_from_csv`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/generate_statistics_from_csv) to compute statistics for our training data. (ignore the snappy warnings)

TFDV can compute descriptive [statistics](https://github.com/tensorflow/metadata/blob/v0.6.0/tensorflow_metadata/proto/v0/statistics.proto) that provide a quick overview of the data in terms of the features that are present and the shapes of their value distributions.

Internally, TFDV uses [Apache Beam](https://beam.apache.org/)'s data-parallel processing framework to scale the computation of statistics over large datasets. For applications that wish to integrate deeper with TFDV (e.g., attach statistics generation at the end of a data-generation pipeline), the API also exposes a Beam PTransform for statistics generation.

In [70]:
train_stats = tfdv.generate_statistics_from_dataframe(TRAIN_DATA)

Now let's use [`tfdv.visualize_statistics`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/visualize_statistics), which uses [Facets](https://pair-code.github.io/facets/) to create a succinct visualization of our training data:

* Notice that numeric features and catagorical features are visualized separately, and that charts are displayed showing the distributions for each feature.
* Notice that features with missing or zero values display a percentage in red as a visual indicator that there may be issues with examples in those features.  The percentage is the percentage of examples that have missing or zero values for that feature.
* Notice that there are no examples with values for `pickup_census_tract`.  This is an opportunity for dimensionality reduction!
* Try clicking "expand" above the charts to change the display
* Try hovering over bars in the charts to display bucket ranges and counts
* Try switching between the log and linear scales, and notice how the log scale reveals much more detail about the `payment_type` categorical feature
* Try selecting "quantiles" from the "Chart to show" menu, and hover over the markers to show the quantile percentages

In [71]:
# docs-infra: no-execute
tfdv.visualize_statistics(train_stats)

<!-- <img class="tfo-display-only-on-site" src="images/statistics.png"/> -->

## Infer a schema

Now let's use [`tfdv.infer_schema`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/infer_schema) to create a schema for our data.  A schema defines constraints for the data that are relevant for ML. Example constraints include the data type of each feature, whether it's numerical or categorical, or the frequency of its presence in the data.  For categorical features the schema also defines the domain - the list of acceptable values.  Since writing a schema can be a tedious task, especially for datasets with lots of features, TFDV provides a method to generate an initial version of the schema based on the descriptive statistics.

Getting the schema right is important because the rest of our production pipeline will be relying on the schema that TFDV generates to be correct.  The schema also provides documentation for the data, and so is useful when different developers work on the same data.  Let's use [`tfdv.display_schema`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/display_schema) to display the inferred schema so that we can review it.

In [72]:
schema = tfdv.infer_schema(statistics=train_stats)
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'qty_105',INT,required,,-
'qty_all',INT,required,,-
'festiva_name',STRING,required,,'festiva_name'
'sale_date',FLOAT,required,,-
'stnpres',FLOAT,required,,-
'seapres',FLOAT,required,,-
'max_temperature',FLOAT,required,,-
'min_temperature',FLOAT,required,,-
'avg_temperature',FLOAT,required,,-


,Values
Domain,
'festiva_name',"'一般', '七夕', '中元節農曆7/15', '中秋節農曆8/15', '佛陀誔辰紀念日', '元宵節農曆1/15', '元旦', '兒童節', '冬至', '勞動節', '國慶日', '婦女節', '平安夜', '情人節', '愚人節', '感恩節', '教師節', '植樹節', '母親節', '清明', '端午節', '耶誕節', '臘八節', '農曆10/1', '農曆10/15', '農曆11/1', '農曆11/15', '農曆12/1', '農曆12/15', '農曆2/1', '農曆2/15', '農曆3/1', '農曆3/15', '農曆4/1', '農曆4/15', '農曆5/1', '農曆5/15', '農曆6/1', '農曆6/15', '農曆7/1', '農曆8/1', '農曆9/1', '農曆9/15', '重陽節', '除夕'"


## Check evaluation data for errors

So far we've only been looking at the training data.  It's important that our evaluation data is consistent with our training data, including that it uses the same schema.  It's also important that the evaluation data includes examples of roughly the same ranges of values for our numerical features as our training data, so that our coverage of the loss surface during evaluation is roughly the same as during training.  The same is true for categorical features.  Otherwise, we may have training issues that are not identified during evaluation, because we didn't evaluate part of our loss surface.

* Notice that each feature now includes statistics for both the training and evaluation datasets.
* Notice that the charts now have both the training and evaluation datasets overlaid, making it easy to compare them.
* Notice that the charts now include a percentages view, which can be combined with log or the default linear scales.
* Notice that the mean and median for `trip_miles` are different for the training versus the evaluation datasets.  Will that cause problems?
* Wow, the max `tips` is very different for the training versus the evaluation datasets.  Will that cause problems?
* Click expand on the Numeric Features chart, and select the log scale.  Review the `trip_seconds` feature, and notice the difference in the max.  Will evaluation miss parts of the loss surface?

In [73]:
# Compute stats for evaluation data
eval_stats = tfdv.generate_statistics_from_dataframe(EVAL_DATA)

In [74]:
# docs-infra: no-execute
# Compare evaluation data with training data
tfdv.visualize_statistics(lhs_statistics=eval_stats, rhs_statistics=train_stats,
                          lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

<!-- <img class="tfo-display-only-on-site" src="images/statistics_eval.png"/> -->

## Check for evaluation anomalies

Does our evaluation dataset match the schema from our training dataset?  This is especially important for categorical features, where we want to identify the range of acceptable values.

Key Point: What would happen if we tried to evaluate using data with categorical feature values that were not in our training dataset?  What about numeric features that are outside the ranges in our training dataset?

In [75]:
# Check eval data for errors by validating the eval data stats using the previously inferred schema.
anomalies = tfdv.validate_statistics(statistics=eval_stats, schema=schema)
tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'festiva_name',Unexpected string values,"Examples contain values missing from the schema: 清明農曆3/15 (<1%), 父親節 (<1%)."


## Fix evaluation anomalies in the schema

Oops!  It looks like we have some new values for `company` in our evaluation data, that we didn't have in our training data.  We also have a new value for `payment_type`.  These should be considered anomalies, but what we decide to do about them depends on our domain knowledge of the data.  If an anomaly truly indicates a data error, then the underlying data should be fixed.  Otherwise, we can simply update the schema to include the values in the eval dataset.

Key Point: How would our evaluation results be affected if we did not fix these problems?

Unless we change our evaluation dataset we can't fix everything, but we can fix things in the schema that we're comfortable accepting.  That includes relaxing our view of what is and what is not an anomaly for particular features, as well as updating our schema to include missing values for categorical features.  TFDV has enabled us to discover what we need to fix.

Let's make those fixes now, and then review one more time.

In [76]:
# Relax the minimum fraction of values that must come from the domain for feature company.
company = tfdv.get_feature(schema, 'festiva_name')
company.distribution_constraints.min_domain_mass = 0.9

# Add new value to the domain of feature payment_type.
# payment_type_domain = tfdv.get_domain(schema, 'payment_type')
# payment_type_domain.value.append('Prcard')

# Validate eval stats after updating the schema 
updated_anomalies = tfdv.validate_statistics(eval_stats, schema)
tfdv.display_anomalies(updated_anomalies)

Hey, look at that!  We verified that the training and evaluation data are now consistent!  Thanks TFDV ;)

## Schema Environments

We also split off a 'serving' dataset for this example, so we should check that too.  By default all datasets in a pipeline should use the same schema, but there are often exceptions. For example, in supervised learning we need to include labels in our dataset, but when we serve the model for inference the labels will not be included. In some cases introducing slight schema variations is necessary.

**Environments** can be used to express such requirements. In particular, features in schema can be associated with a set of environments using `default_environment`, `in_environment` and `not_in_environment`.

For example, in this dataset the `tips` feature is included as the label for training, but it's missing in the serving data. Without environment specified, it will show up as an anomaly.

In [78]:
serving_stats = tfdv.generate_statistics_from_dataframe(SERVING_DATA)
serving_anomalies = tfdv.validate_statistics(serving_stats, schema)

tfdv.display_anomalies(serving_anomalies)

We'll deal with the `tips` feature below.  We also have an INT value in our trip seconds, where our schema expected a FLOAT. By making us aware of that difference, TFDV helps uncover inconsistencies in the way the data is generated for training and serving. It's very easy to be unaware of problems like that until model performance suffers, sometimes catastrophically. It may or may not be a significant issue, but in any case this should be cause for further investigation.

In this case, we can safely convert INT values to FLOATs, so we want to tell TFDV to use our schema to infer the type.  Let's do that now.

In [80]:
options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)
serving_stats = tfdv.generate_statistics_from_dataframe(SERVING_DATA, stats_options=options)
serving_anomalies = tfdv.validate_statistics(serving_stats, schema)

tfdv.display_anomalies(serving_anomalies)

Now we just have the `tips` feature (which is our label) showing up as an anomaly ('Column dropped').  Of course we don't expect to have labels in our serving data, so let's tell TFDV to ignore that.

In [82]:
# All features are by default in both TRAINING and SERVING environments.
schema.default_environment.append('TRAINING')
schema.default_environment.append('SERVING')

# Specify that 'tips' feature is not in SERVING environment.
# tfdv.get_feature(schema, 'tips').not_in_environment.append('SERVING')

# serving_anomalies_with_env = tfdv.validate_statistics(
#     serving_stats, schema, environment='SERVING')

# tfdv.display_anomalies(serving_anomalies_with_env)

## Check for drift and skew

In addition to checking whether a dataset conforms to the expectations set in the schema, TFDV also provides functionalities to detect drift and skew.  TFDV performs this check by comparing the statistics of the different datasets based on the drift/skew comparators specified in the schema.

### Drift

Drift detection is supported for categorical features and between consecutive spans of data (i.e., between span N and span N+1), such as between different days of training data.  We express drift in terms of [L-infinity distance](https://en.wikipedia.org/wiki/Chebyshev_distance), and you can set the threshold distance so that you receive warnings when the drift is higher than is acceptable.  Setting the correct distance is typically an iterative process requiring domain knowledge and experimentation.

### Skew

TFDV can detect three different kinds of skew in your data - schema skew, feature skew, and distribution skew.

#### Schema Skew

Schema skew occurs when the training and serving data do not conform to the same schema. Both training and serving data are expected to adhere to the same schema. Any expected deviations between the two (such as the label feature being only present in the training data but not in serving) should be specified through environments field in the schema.

#### Feature Skew

Feature skew occurs when the feature values that a model trains on are different from the feature values that it sees at serving time. For example, this can happen when:

* A data source that provides some feature values is modified between training and serving time
* There is different logic for generating features between training and serving. For example, if you apply some transformation only in one of the two code paths.

#### Distribution Skew

Distribution skew occurs when the distribution of the training dataset is significantly different from the distribution of the serving dataset. One of the key causes for distribution skew is using different code or different data sources to generate the training dataset. Another reason is a faulty sampling mechanism that chooses a non-representative subsample of the serving data to train on.

In [83]:
# Add skew comparator for 'payment_type' feature.
# payment_type = tfdv.get_feature(schema, 'payment_type')
# payment_type.skew_comparator.infinity_norm.threshold = 0.01

# Add drift comparator for 'company' feature.
company=tfdv.get_feature(schema, 'festiva_name')
company.drift_comparator.infinity_norm.threshold = 0.001

skew_anomalies = tfdv.validate_statistics(train_stats, schema,
                                          previous_statistics=eval_stats,
                                          serving_statistics=serving_stats)

tfdv.display_anomalies(skew_anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'festiva_name',High Linfty distance between current and previous,"The Linfty distance between current and previous is 0.00337268 (up to six significant digits), above the threshold 0.001. The feature value with maximum difference is: 農曆8/1"


In this example we do see some drift, but it is well below the threshold that we've set.

## Freeze the schema

Now that the schema has been reviewed and curated, we will store it in a file to reflect its "frozen" state.

In [84]:
from tensorflow.python.lib.io import file_io
from google.protobuf import text_format

file_io.recursive_create_dir(OUTPUT_DIR)
schema_file = os.path.join(OUTPUT_DIR, 'schema.pbtxt')
tfdv.write_schema_text(schema, schema_file)

!cat {schema_file}

feature {
  name: "qty_105"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "qty_all"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "festiva_name"
  type: BYTES
  domain: "festiva_name"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  distribution_constraints {
    min_domain_mass: 0.9
  }
  drift_comparator {
    infinity_norm {
      threshold: 0.001
    }
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "sale_date"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "stnpres"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "seapres"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
  

## When to use TFDV

It's easy to think of TFDV as only applying to the start of your training pipeline, as we did here, but in fact it has many uses.  Here's a few more:

* Validating new data for inference to make sure that we haven't suddenly started receiving bad features
* Validating new data for inference to make sure that our model has trained on that part of the decision surface
* Validating our data after we've transformed it and done feature engineering (probably using [TensorFlow Transform](https://www.tensorflow.org/tfx/transform/get_started)) to make sure we haven't done something wrong